In [1]:
import pandas as pd

file_in = "CC_Classified_DAOPHOT_June192023.csv"
csv = pd.read_csv(file_in)

csv.columns

Index(['RA', 'DEC', 'f090w', 'f187n', 'f200w', 'f335m', 'f444w', 'f470n',
       'f090w-f444w', 'f090w-f187n', 'δ_f090w-f187n', 'f187n-f200w',
       'δ_f187n-f200w', 'f200w-f335m', 'δ_f200w-f335m', 'f335m-f444w',
       'δ_f335m-f444w', 'f444w-f470n', 'δ_f444w-f470n',
       'δ_(f090w-f200w)-(f200w-f444w)', 'Sum1', 'e_f090w', 'e_f187n',
       'e_f200w', 'e_f335m', 'e_f444w', 'e_f470n', 'e_f090w-f444w',
       'e_f090w-f187n', 'e_δ_f090w-f187n', 'e_f187n-f200w', 'e_δ_f187n-f200w',
       'e_f200w-f335m', 'e_δ_f200w-f335m', 'e_f335m-f444w', 'e_δ_f335m-f444w',
       'e_f444w-f470n', 'e_δ_f444w-f470n', 'e_δ_(f090w-f200w)-(f200w-f444w)',
       'e_Sum1', 'Class_PRF', 'Prob_PRF', 'Class_RF', 'Prob_RF', 'Init_Class',
       'Survey', 'SPICY_ID'],
      dtype='object')

In [4]:
columns = ['RA', 'DEC', 'f090w', 'f187n','f200w', 'f335m', 'f444w','f470n', 'Prob_PRF','Prob_RF']

new_cols = ['RA', 'DEC', 'F090w','F187n','F200w','F335m','F444w','F470n',  'Prob YSO PRF', 'Prob YSO RF'] #'F770w', 'F1130w', 'F1280w', 'F1800w',


In [50]:

file_out = file_in.split('.')[0]+'_latex.txt'
print("Printing to ", file_out)
f = open(file_out,'w')

for j in range(0,len(columns)):
    f.write(new_cols[j]+'&')
f.write('\\\ \n')
for i in range(0,len(csv)):
    for j in range(0,len(columns)):
        str_tmp = str("%.4f" %csv[columns[j]][i])+"&"
        f.write(str_tmp)
    f.write('\\\ \n')

f.close()

Printing to  CC_Webb_Predictions_Prob_2_5sigFeb172023_latex.txt


In [5]:
csv_yso = csv[(csv.Class_RF==0)|(csv.Class_PRF==0)].reset_index()
csv_yso = csv_yso.loc[csv_yso.isnull().sum(1).sort_values(ascending=1).index,columns]


file_out = file_in.split('.')[0]+'_latex_ysos.txt'
print("Printing to ", file_out)


f = open(file_out,'w')

for j in range(0,len(columns)):
    f.write(new_cols[j]+'&')
f.write('\\hline \\\ \n')
for i in range(0,len(csv_yso)):
    for j in range(0,len(columns)):
        str_tmp = str("%.4f" %csv_yso[columns[j]].values[i])+"&"
        f.write(str_tmp)
    f.write('\\\ \n')

f.close()

Printing to  CC_Classified_DAOPHOT_June192023_latex_ysos.txt
